# L'importance des modèles

## Données

Repartons des données que nous avons mis en forme, et amenons progressivement une analyse plus avancée

In [5]:
import pandas as pd
df = pd.read_excel("../data/data.xlsx")
df.head()

,Unnamed: 0,id,title,publication_year,abstract
0,0,https://openalex.org/W2159397589,Computational Social Science,2009.0,A field is emerging that leverages the capacit...
1,1,https://openalex.org/W2070907364,Manifesto of computational social science,2012.0,NaN
2,2,https://openalex.org/W3081158114,Computational social science: Obstacles and op...,2020.0,"Data sharing, research ethics, and incentives ..."
3,3,https://openalex.org/W3022499311,Computational Social Science and Sociology,2020.0,The integration of social science with compute...
4,4,https://openalex.org/W3174174150,Integrating explanation and prediction in comp...,2021.0,NaN


## Spacy pour aller vers les modèles

Installer Spacy et un modèle adapté

In [4]:
#pip install numpy==1.26

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 2.5 MB/s  0:00:08m0:00:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.4.2
    Uninstalling numpy-2.4.2:
      Successfully uninstalled numpy-2.4.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scipy 1.17.0 requires numpy<2.7,>=1.26.4, but you have numpy 1.26.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [28]:
#!pip install -U spacy
# !python -m spacy download en_core_web_trf

In [1]:
import spacy

pipeline = spacy.load("en_core_web_trf")

phrase = "Computational social sciences are great. I like to program in Python to do Sociology."

doc = pipeline(phrase)

Disabling PyTorch because PyTorch >= 2.4 is required but found 2.2.2
PyTorch was not found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [3]:
doc

Computational social sciences are great. I like to program in Python to do Sociology.

In [4]:
type(doc)

spacy.tokens.doc.Doc

Le texte est "structuré" : composé de différents éléments qui n'existaient pas avant

In [5]:
doc[2]

sciences

POS, lemmatisation, etc.

In [6]:
for token in doc:
    print(token.text, token.lemma_, token.pos_, token.dep_)

Computational computational ADJ amod
social social ADJ amod
sciences science NOUN nsubj
are be AUX ROOT
great great ADJ acomp
. . PUNCT punct
I I PRON nsubj
like like VERB ROOT
to to PART aux
program program VERB xcomp
in in ADP prep
Python Python PROPN pobj
to to PART aux
do do VERB advcl
Sociology sociology NOUN dobj
. . PUNCT punct


Récupérer uniquement les verbes ?

In [7]:
[word for word in doc if word.pos_ == "ADJ"]

[Computational, social, great]

NER

In [8]:
for ent in doc.ents:
    print(ent.text, ent.label_)

Python PRODUCT


In [14]:
result = df["abstract"][0:10].dropna().apply(pipeline)

In [23]:
for ent in result.iloc[3].ents:
    print(ent.text, ent.label_)

zero CARDINAL
zero CARDINAL
zero CARDINAL
13 CARDINAL
25 CARDINAL
English LANGUAGE
today DATE
two CARDINAL
1 CARDINAL
zero CARDINAL
2 CARDINAL


In [25]:
from spacy import displacy
displacy.render(result.iloc[3], style="ent", 
                jupyter=True)

Plongements

In [36]:
# Compare two documents
doc1 = pipeline(df["abstract"][15])
doc2 = pipeline(df["abstract"][56])
print(doc1.similarity(doc2))

0.0


/var/folders/f9/d2d_05ws5gncmml0fx0c00kw0000gp/T/ipykernel_30528/1730970218.py:4: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  print(doc1.similarity(doc2))
/var/folders/f9/d2d_05ws5gncmml0fx0c00kw0000gp/T/ipykernel_30528/1730970218.py:4: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  print(doc1.similarity(doc2))


## Utiliser des modèles d'HuggingFace : le cas de GliNER

Aller vers les modèles :
- HuggingFace
- Cartes des modèles
- Transformers

Un modèle plus complexe pour l'identification d'entités

Par exemple : https://huggingface.co/urchade/gliner_multi_pii-v1

In [1]:
#!pip install gliner

In [1]:
from gliner import GLiNER

model = GLiNER.from_pretrained("gliner-community/gliner_small-v2.5", load_tokenizer=True)

Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

In [3]:
text = "Émilien is fund of computational social sciences"

labels = ["type of activity"]

entities = model.predict_entities(text, labels)

entities

[{'start': 19,
  'end': 48,
  'text': 'computational social sciences',
  'label': 'type of activity',
  'score': 0.7678732872009277}]

In [10]:
result = df[df["abstract"].notna()]["abstract"][:20].apply(lambda x:  model.predict_entities(x, ["methods"]))

In [13]:
for i in result:
    print(i)

[]
[{'start': 0, 'end': 12, 'text': 'Data sharing', 'label': 'methods', 'score': 0.8989892601966858}, {'start': 14, 'end': 29, 'text': 'research ethics', 'label': 'methods', 'score': 0.8246647119522095}]
[{'start': 162, 'end': 183, 'text': 'computational methods', 'label': 'methods', 'score': 0.531370222568512}, {'start': 385, 'end': 406, 'text': 'bibliometric analysis', 'label': 'methods', 'score': 0.9459057450294495}, {'start': 411, 'end': 428, 'text': 'in-depth analysis', 'label': 'methods', 'score': 0.5802873969078064}, {'start': 508, 'end': 531, 'text': 'social network analysis', 'label': 'methods', 'score': 0.9138570427894592}]
[{'start': 289, 'end': 317, 'text': 'computational social science', 'label': 'methods', 'score': 0.5637627840042114}, {'start': 647, 'end': 661, 'text': 'classification', 'label': 'methods', 'score': 0.6134197115898132}, {'start': 798, 'end': 808, 'text': 'generation', 'label': 'methods', 'score': 0.5292314291000366}]
[{'start': 229, 'end': 257, 'text': 'C

## Application

Faire de l'analyse de sentiments

Une question : **quelles sont les prises de paroles les plus négatives ?**

- Embarras du choix
    - Par ex : [🚀 distilbert-based Multilingual Sentiment Classification Model
](https://huggingface.co/tabularisai/multilingual-sentiment-analysis)
- Comprendre le modèle / ce qu'il fait
- Importance d'évaluer son résultat